In [ ]:
!gdown 1G2DkYk-vpbTOZJhpgOm9Sj1peYnLqvie

In [ ]:
!pip install torch torchvision albumentations pandas

In [ ]:

import zipfile
import os

zip_path = "/content/cropped_dataset_4k_face.zip"
extract_path = "/content/data"  # nơi chứa dữ liệu sau giải nén

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Kiểm tra kết quả
print("File sau khi giải nén:")
print(os.listdir(extract_path))


In [31]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import os
from PIL import Image

class MyDataset(Dataset):
    def __init__(self, root, labels_csv, transform=None):
        self.root = root
        df = pd.read_csv(labels_csv)
        self.items = df[['filename', 'label']].values

        # Tạo ánh xạ label → chỉ số
        unique_labels = sorted(df['label'].unique())
        self.label2index = {name: i for i, name in enumerate(unique_labels)}

        self.transform = transform

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        fn, label_str = self.items[idx]
        label_idx = self.label2index[label_str]
        img_path = os.path.join(self.root, label_str, fn)
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, label_idx



transform = transforms.Compose([
    transforms.RandomResizedCrop(512),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std=[0.229,0.224,0.225]),
])
dataset = MyDataset('/content/data/Dog Emotion', '/content/data/Dog Emotion/labels.csv', transform)
loader = DataLoader(dataset, batch_size=8, shuffle=True, num_workers=2)


In [ ]:
import torch
import importlib
import pure34
import torch.nn as nn
from pure34 import PURe34
importlib.reload(pure34)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = PURe34(num_classes=4).to(device)
opt = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=20)
crit = nn.CrossEntropyLoss()

for epoch in range(20):
    model.train()
    total, correct = 0,0
    try:
      for imgs, labels in loader:
          imgs, labels = imgs.to(device), labels.to(device)
          opt.zero_grad()
          logits = model(imgs)
          loss = crit(logits, labels)
          loss.backward()
          opt.step()

          pred = logits.argmax(dim=1)
          total += labels.size(0)
          correct += (pred==labels).sum().item()
    except FileNotFoundError:
        print('error')
        continue
    sched.step()
    print(f'Epoch {epoch+1}  Acc={correct/total:.4f}  LR={sched.get_last_lr()[0]:.4f}')
